In [1]:
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin

In [3]:
bank_file = '../Bank_list.xlsx'
banks = pd.read_excel(bank_file)
names = banks['Name'].unique().tolist()

In [2]:
session = requests.Session()
session.headers["User-Agent"] = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"

In [42]:
for bank in tqdm(names):
    for client_type in ['Individual', 'Corporation']:
        folder_location = f'../bank_data/{client_type}/{bank}'
        if not os.path.exists(folder_location):
            os.makedirs(folder_location)
        urls = banks[banks['Name'] == bank][client_type].dropna().tolist() 
        for url in urls:
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, "html.parser")
                    for link in soup.select("a[href$='.pdf']"):
                        try:
                            # Name the pdf files using the last portion of each link which is unique in this case
                            filename = os.path.join(folder_location, link['href'].split('/')[-1])
                            with open(filename, 'wb') as f:
                                pdf_content = requests.get(urljoin(url, link['href'])).content
                                f.write(pdf_content)
                        except (requests.RequestException, IOError) as e:
                            print(f"Error occurred while writing the file: {e}")
            except requests.RequestException as e:
                print(f"Error occurred while fetching URL: {e}")

                #downloading websites
                resp = requests.get(url)
                html_content = resp.text

                # Extract filename from URL or use a unique identifier
                filename = url.split('/')[-1]  # Adjust this based on the URL structure
                
                # Modify the filename to make it unique within the directory
                filepath = os.path.join(folder_location, f"{filename}.html")

                # Write the HTML content to a file
                with open(filepath, 'w', encoding='utf-8') as f:
                    f.write(html_content)  # Write the raw HTML content directly to the file

100%|██████████| 19/19 [00:29<00:00,  1.55s/it]
